In [1]:
import base64
import numpy as np
import io
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, load_model
from keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array
from flask import request
from flask import jsonify
from flask import Flask

Using TensorFlow backend.


In [2]:
app = Flask(__name__)

In [3]:
def get_model():
    global model
    model = load_model('VGG16_cats_and_dogs.h5')
    model._make_predict_function()
    print(" * Model loaded!")

In [5]:
def preprocess_image(image, target_size):
    if image.mode != "RGB":
        image = image.convert("RGB")
    image = image.resize(target_size)
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    return image

In [6]:
print(" * Loading Keras model...")
get_model()
graph = tf.get_default_graph()

 * Loading Keras model...
 * Model loaded!


/usr/local/anaconda3/envs/keras_env/lib/python3.7/site-packages/keras/engine/saving.py:384: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [7]:
model.predict():
    with graph.as_default():
        @app.route("/predict", methods=["POST"])
        def predict():
            message = request.get_json(force=True)
            encoded = message['image']
            decoded = base64.b64decode(encoded)
            image = Image.open(io.BytesIO(decoded))
            processed_image = preprocess_image(image, target_size=(224, 224))

            prediction = model.predict(processed_image).tolist()

            response = {
                'prediction': {
                    'dog': prediction[0][0],
                    'cat': prediction[0][1]
                }
            }
            return jsonify(response)